In [4]:
%load_ext autoreload
%autoreload 2
import os
import matplotlib.pyplot as plt
import seaborn as sns
from os.path import join
from tqdm import tqdm
import pandas as pd
import sys
from IPython.display import display, HTML
from typing import List
import numpy as np
import matplotlib
import imodelsx.util
from copy import deepcopy
import re
# import sasc.notebook_helper
import sasc.viz
import scipy.special
from collections import defaultdict
from pprint import pprint
import joblib
# import viz
from sasc.config import RESULTS_DIR

MAIN_DIR = join(RESULTS_DIR, 'pilot_v1', 'interactions')
EXPT_DIRS = sorted([join(MAIN_DIR, dir_name) for dir_name in os.listdir(MAIN_DIR)])
EXPT_DIR = EXPT_DIRS[0]
rows = joblib.load(join(EXPT_DIR, f'rows.pkl'))
expls = rows.expl.values
prompts_paragraphs = joblib.load(
    join(EXPT_DIR, "prompts_paragraphs.pkl"),
)
prompts = prompts_paragraphs["prompts"]
paragraphs = prompts_paragraphs["paragraphs"]
# paragraphs = rows.paragraph.values
# prompts = rows.prompt.values
# voxel_nums = rows.module_num.values
# subjects = rows.subject.values

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Heatmaps

**Explanation** <> Story Heatmap

In [5]:
for EXPT_DIR in EXPT_DIRS[:1]:
    scores_data = joblib.load(join(EXPT_DIR, f"scores_mod_ngram_length={0}.pkl"))
    s = scores_data['scores_mean'].T
    # s = scipy.special.softmax(s, axis=1)
    # s = (s - s.min()) / (s.max() - s.min())
    rows = joblib.load(join(EXPT_DIR, f'rows1_rep.pkl'))
    expls = rows.expl.values
    sasc.viz.heatmap(s, expls, ylab='Story', xlab='Explanation')
    # plt.savefig(join(RESULTS_DIR, 'stories', f'{EXPT_NAME}_story_data_match.png'), dpi=300)
    # plt.savefig(join(RESULTS_DIR, 'stories', f'{EXPT_NAME}_story_data_match.pdf'), bbox_inches='tight')
    # plt.tight_layout()
    # plt.show()
# sasc.viz.save_figs_to_single_pdf('../results/expl_story_match.pdf')

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│    1 for EXPT_DIR in EXPT_DIRS[:1]:                                                              │
│ ❱  2 │   scores_data = joblib.load(join(EXPT_DIR, f"scores_mod_ngram_length={0}.pkl"))           │
│    3 │   s = scores_data['scores_mean'].T                                                        │
│    4 │   # s = scipy.special.softmax(s, axis=1)                                                  │
│    5 │   # s = (s - s.min()) / (s.max() - s.min())                                               │
│                                                                                                  │
│ /home/chansingh/imodelsx/.venv/lib/python3.11/site-packages/joblib/numpy_pickle.py:650 in load   │
│                                                                                                  │
│   647 │   │   with _read_fileobject(fobj, filename, mmap_mode) as fobj:                          │
│   648 │   │   │   obj = _unpickle(fobj)                                                          │
│   649 │   else:                                                                                  │
│ ❱ 650 │   │   with open(filename, 'rb') as f:                                                    │
│   651 │   │   │   with _read_fileobject(f, filename, mmap_mode) as fobj:                         │
│   652 │   │   │   │   if isinstance(fobj, str):                                                  │
│   653 │   │   │   │   │   # if the returned file object is a string, this means we               │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
FileNotFoundError: [Errno 2] No such file or directory: 
'/home/chansingh/automated-explanations/results/pilot_v1/interactions/uts02___jun14___seed=1/scores_mod_ngram_lengt
h=0.pkl'

**Module** <> Story Heatmap

In [ ]:
for EXPT_DIR in EXPT_DIRS[:1]:
    # keys: scores_mod, scores_max_mod, all_scores, all_ngrams
    # ngram_lengths = [10, 50, 100, 384]
    ngram_lengths = [0]
    for ngram_length in ngram_lengths:
        scores_mod_dict = joblib.load(join(EXPT_DIR, f'scores_mod_ngram_length={ngram_length}.pkl'))
        rows = joblib.load(join(EXPT_DIR, f'rows1_rep.pkl'))
        expls = rows.expl.values

        s = scores_mod_dict['scores_mean'].T
        s = scipy.special.softmax(s, axis=0)
        # s = (s - s.min()) / (s.max() - s.min())
        sasc.viz.heatmap(s, expls, ylab='Story', xlab='Module (fit to fMRI voxel)', clab='Mean module response')

        n = s.shape[0]
        # plt.plot([0, n], [0, n], '--', color='gray', alpha=0.1)
        plt.title(f'Model window: {ngram_length} words', fontsize='small')
        plt.savefig(join(EXPT_DIR, f'mod_story_match_ngram_length={ngram_length}.png'), dpi=300)
        plt.savefig(join(EXPT_DIR, f'mod_story_match_ngram_length={ngram_length}.png'), bbox_inches='tight')
        # plt.show()
# sasc.viz.save_figs_to_single_pdf('../results/expl_mod_match.pdf')

### Best voxels automated

In [ ]:
vox_diffs = defaultdict(list)
for EXPT_DIR in EXPT_DIRS:
    ngram_length = 0
    scores_mod_dict = joblib.load(join(EXPT_DIR, f'scores_mod_ngram_length={ngram_length}.pkl'))
    rows = joblib.load(join(EXPT_DIR, f'rows.pkl'))
    expls = rows.expl.values


    s = scores_mod_dict['scores_mean'].T
    s = scipy.special.softmax(s, axis=0)

    BLOCK_SIZE = 4
    for i in range(0, rows.shape[0]):
        row = rows.iloc[i]
        s_row = s[i]
        block_idxs = range((i // BLOCK_SIZE) * BLOCK_SIZE, (i // BLOCK_SIZE + 1) * BLOCK_SIZE)
        diff = s_row[i] - np.mean([s_row[j] for j in range(s_row.shape[0]) if j not in block_idxs])
        vox_diffs[(row.module_num, row.expl)].append(diff)
vox_diffs = {k: np.mean(v) for k, v in vox_diffs.items()}

In [ ]:
# display all rows
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(pd.DataFrame(vox_diffs.items()).sort_values(1, ascending=False).head(30))